# <center> <img src="figs/LogoUFSCar.jpg" alt="Logo UFScar" width="110" align="left"/>  <br/> <center>Universidade Federal de São Carlos (UFSCar)<br/><font size="4"> Departamento de Computação, campus Sorocaba</center></font>
</p>

<br/>
<font size="4"><center><b>Disciplina: Aprendizado de Máquina</b></center></font>
  
<font size="3"><center>Prof. Dr. Tiago A. Almeida</center></font>

<br/>
<center><i><b>
Atenção: não são autorizadas cópias, divulgações ou qualquer tipo de uso deste material sem o consentimento prévio dos autores.
</center></i></b>
<br/>

# <center>Exercício - Naive Bayes</center>

Neste exercício, você implementará o método Naive Bayes e verá como ele utiliza os dados para fazer classificações de amostras não vistas. Antes de começar este exercício, é recomendável que você revise os conceitos apresentados em aula.

### Descrição do problema
Rodonildo é um jogador nato de *League of Legends*, um jogo de estratégia que envolve a batalha entre dois times (para maiores detalhes, consulte http://br.leagueoflegends.com/), e esteve coletando dados nas partidas em que jogou. O objetivo de Rodonildo é prever o vencedor de uma determinada batalha a partir de algumas informações. Na coleta de dados que Rodonildo fez, ele utilizou amostras compostas pelos 5 atributos binários (*1 = sim* e *0 = não*) a seguir:

1. *primeiroAbate*: indica se a primeira morte do jogo foi realizada pelo time de Rodonildo;
2. *primeiraTorre*: indica se a primeira torre destruída do jogo foi derrubada pelo time de Rodonildo (Figura 1a);
3. *primeiroInibidor*: indica se o primeiro inibidor destruído do jogo foi derrubado pelo time de Rodonildo (Figura 1b);
4. *primeiroDragao*: indica se o personagem Dragão foi abatido primeiro pelo time de Rodonildo (Figura 1c);
4. *primeiroBaron*: indica se o personagem Baron foi abatido primeiro pelo time de Rodonildo (Figura 1d).

<div style="display:inline-block;">
    <div>
    <div style="padding: 5px; float: left;">
        <img src="figs/turret.png" style="height:180px;"/>
        <center><em>(a) Torre</em></center>
    </div>
    <div style="padding: 5px; float: left;">
        <img src="figs/inhibitor.png"  style="height:180px;"/> 
        <center><em>(b) Inibidor</em></center>
    </div>
    <div style="padding: 5px; float: left;">
        <img src="figs/dragon.png"  style="height:180px;"/>
        <center><em>(c) Dragão</em></center>
    </div>
    <div style="padding: 5px; float: left;">
        <img src="figs/baron.png"  style="height:180px;"/>
        <center><em>(d) Baron</em></center>
    </div>
    </div>
    <center><em>Figura 1. Objetos e Criaturas de League of Legends.</em></center>
</div> 

Por exemplo, a amostra $x = [0, 0, 1, 1, 0]$ e $y = 0$  representa um jogo no qual o time de Rodonildo destruiu primeiro um inibidor inimigo e derrotou o dragão antes da equipe inimiga. Por sua vez, a equipe adversária fez o primeiro abate do jogo, destruiu a primeira torre e derrotou o Baron. Essa partida foi vencida pela equipe adversária.

Após longo período de coleta de dados, Rodonildo precisa da sua ajuda para prever o resultado de outras partidas utilizando as informações armazenadas. A sua função é implementar o classificador Naive Bayes para predizer qual será o resultado das próximas partidas de Rodonildo, condicionado aos valores dos atributos.

Instruções
----------

Este arquivo contém o código que auxiliará no desenvolvimento do exercício. Você precisará completar as seguintes funções:

* calculaProbabilidade()
* classificacao()

Você não poderá criar nenhuma outra função. Apenas altere as rotinas fornecidas.

## Parte 1: Calcular as probabilidades

Nesta etapa, você precisará implementar a função *calcularProbabilidades()*. Esta função retornará os vetores com as probabilidades de cada atributo para as classes.

Primeiro, vamos carregar a base de dados com as partidas jogadas pelo Rodonildo.

In [1]:
# -*- coding: utf-8 -*-

import numpy as np #importa a biblioteca usada para trabalhar com vetores de matrizes
import pandas as pd #importa a biblioteca usada para trabalhar com dataframes (dados em formato de tabela) e análise de dados

# Importa o arquivo e guarda em um dataframe do Pandas
df_dataset = pd.read_csv( 'dados.csv', sep=',', index_col=None)

print('Dados carregados com sucesso!')

Dados carregados com sucesso!


Agora, vamos dar uma olhada nas 6 primeiras amostras da base de dados.

In [2]:
# vamos usar a função display para imprimir o dataframe, pois deixa mais bonito. 
# Mas, também poderíamos ter usado a função print: print(df_dataset.head(n=6))
display(df_dataset.head(n=6))

,primAbate,primTorre,primInibidor,primDragao,primBaron,Classe
0,0,0,1,1,0,0
1,0,1,1,0,1,1
2,1,1,1,0,0,1
3,0,0,0,0,0,0
4,1,1,1,0,1,1
5,0,1,1,1,0,1


Vamos guardar os dados dentro de uma matriz e as classes dentro de um vetor.

In [3]:
# pega os valores das n-1 primeiras colunas e guarda em uma matrix X
X = df_dataset.iloc[:, 0:-1].values 

# pega os valores da última coluna e guarda em um vetor Y
Y = df_dataset.iloc[:, -1].values 

# imprime as 5 primeiras linhas da matriz X
display('X:', X[0:5,:])

# imprime os 5 primeiros valores de Y
print('Y:', Y[0:5])

'X:'

array([[0, 0, 1, 1, 0],
       [0, 1, 1, 0, 1],
       [1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1]])

Y: [0 1 1 0 1]


Vamos calcular qual a probabilidade de ocorrência de cada classe.

In [4]:
# Probabilidade das Classes
pVitoria = sum(Y==1)/len(Y) 
pDerrota = sum(Y==0)/len(Y)

print('Probabilidade da classe ser 1 (vitória): %1.2f%%' %(pVitoria*100))
print('Probabilidade da classe ser 0 (derrota): %1.2f%%' %(pDerrota*100))

Probabilidade da classe ser 1 (vitória): 54.00%
Probabilidade da classe ser 0 (derrota): 46.00%


Agora, crie a função que irá calcular as probabilidades de ocorrência de cada atributo em cada classe.

In [5]:
def calcularProbabilidades(X, Y):
    """
    CALCULARPROBABILIDADES Computa a probabilidade de ocorrencia de cada 
    atributo por rotulo possivel. A funcao retorna dois vetores de tamanho n
    (qtde de atributos), um para cada classe.
    
    CALCULARPROBABILIDADES(X, Y) calcula a probabilidade de ocorrencia de cada atributo em cada classe. 
    Cada vetor de saida tem dimensao (n x 1), sendo n a quantidade de atributos por amostra.
    """
    
    #  inicializa os vetores de probabilidades
    pAtrVitoria = np.zeros(X.shape[1])
    pAtrDerrota = np.zeros(X.shape[1])
   ########################## COMPLETE O CÓDIGO AQUI  ########################
    #  Instrucoes: Complete o codigo para encontrar a probabilidade de
    #                ocorrencia de um atributo para uma determinada classe.
    #                Ex.: para a classe 1 (vitoria), devera ser computada um
    #                vetor pAtrVitoria (n x 1) contendo n valores:
    #                P(Atributo1=1|Classe=1), ..., P(Atributo5=1|Classe=1), e o
    #                mesmo para a classe 0 (derrota):
    #                P(Atributo1=1|Classe=0), ..., P(Atributo5=1|Classe=0).
    # 
    
    #TX = X.transpose()
    #print(TX.sum(axis = 1))
    
    #print(X.sum(axis = 0))
    #print(X.shape[0])
    
    
    #PA = X.sum(axis = 0)/X.shape[0]
    #PC1 = Y.sum()/Y.size
    #PC0 = 1 - PC1

    #print(PA)
    #print(PC1A)
    #print(PC1)
    #pAtrVitoria = (PC1A * PA)/PC1
    #pAtrDerrota = (PC0A * PA)/PC0
    
    pAtrVitoria = (X.T * Y).sum(axis = 1)/(Y.sum())
    pAtrDerrota = (X.T * (1 - Y)).sum(axis = 1)/(Y.size - Y.sum())
          
   # print(pAtrVitoria)
    #print(pAtrDerrota)
    
    
    
    
    
    
    
    
    
    
    
    
    ##########################################################################

    return pAtrVitoria, pAtrDerrota

pAtrVitoria, pAtrDerrota = calcularProbabilidades(X,Y)

print('A probabilidade esperada para P(PrimeiroAbate=1|Classe=1) = %.2f%%' %52.96)
print('\nEssa mesma probabilidade calculada no seu codigo foi = %.2f%%' %(pAtrVitoria[0]*100))

A probabilidade esperada para P(PrimeiroAbate=1|Classe=1) = 52.96%

Essa mesma probabilidade calculada no seu codigo foi = 52.96%


## Parte 2: Classificação da própria base usando o método Naive Bayes

Nesta etapa, é realizada a classificação das amostras com base nas probabilidades encontradas no passo anterior. A classificação é realizada verificando se a amostra em questão tem maior probabilidade de pertencer à classe 1 ou à classe 0. Para calcular a probabilidade de uma amostra pertencer a uma determinada classe, é necessário utilizar as probabilidades de ocorrências de atributos previamente computadas. O cálculo pode ser expresso como:

$$ P(y_j|\vec{x}) = \hat{P}(y_{j}) \prod_{x_i \in \vec{x}} \hat{P}(x_{i} | y_{j}) $$

Portanto, a probabilidade de uma amostra $\vec{x}$ pertencer a uma classe $j$ é obtida a partir da probabilidade geral da classe $j$ ($\hat{P}(y_{j})$) multiplicada pelo produtório da probabilidade de ocorrência de cada atributo $x_i$ com relação a classe $j$ ($\hat{P}(x_{i} | y_{j})$).

Se a rotina de classificação estiver correta, espera-se que a acurácia obtida ao classificar a própria base de amostras de jogos que Ronildo participou seja aproximadamente 76,60%. 

Você deverá completar a função **Classificacao()**.

In [6]:
def classificacao(x,pVitoria,pDerrota,pAtrVitoria,pAtrDerrota):
    """
    Classifica se a entrada x pertence a classe 0 ou 1 usando
    as probabilidades extraidas da base de treinamento. Essa funcao 
    estima a predicao de x atraves da maior probabilidade da amostra  
    pertencer a classe 1 ou 0. Tambem retorna as probabilidades condicionais
    de vitoria e derrota, respectivamente.
    
    """

    #  inicializa a classe e as probabilidades condicionais
    classe = 0;
    probVitoria= 0;
    probDerrota = 0;

    ########################## COMPLETE O CÓDIGO AQUI  ########################
    #  Instrucoes: Complete o codigo para estimar a classificacao da amostra
    #                usando as probabilidades extraidas da base de treinamento.
    #                Voce precisara encontrar as probabilidades Bayesianas 
    #                probVitoria p(classe=1|x) e probDerrota p(classe=0|x) e
    #                selecionar a maior.
    # 


    
    probVitoria = pVitoria * np.prod((pAtrVitoria * x) - ((1-pAtrVitoria) * (x-1)))
    probDerrota = np.prod((pAtrDerrota * x) - ((1-pAtrDerrota) * (x-1))) * pDerrota
    
    
    
    
    #print(pAtrVitoria * x)
    #print(probDerrota)
    
    if probVitoria.max() >= probDerrota.max() :
        classe = 1
    
    
    
    
    
    
    
    
    

    ########################################################################## 

    return classe, probVitoria, probDerrota 

resultados = np.zeros( X.shape[0] )

for i in range(X.shape[0]):
    resultados[i], probVitoria, probDerrota = classificacao( X[i,:],pVitoria,pDerrota,pAtrVitoria,pAtrDerrota )

# calcular acuracia
acuracia = np.sum(resultados==Y)/len(Y)

print('\n\nAcuracia esperada para essa base = %.2f%%\n' %76.60);
print('Acuracia obtida pelo seu classificador foi = %.2f%%\n' %( acuracia*100 ) )



Acuracia esperada para essa base = 76.60%

Acuracia obtida pelo seu classificador foi = 76.60%



## Parte 3: Predizendo a classe de novos dados

Já que toda a etapa de treinamento e classificação está concluída, o último passo é permitir que novas amostras sejam classificadas. 

In [16]:
x1_novo = np.array([0,1,0,0,0])

classe, probVitoria, probDerrota = classificacao( x1_novo,pVitoria,pDerrota,pAtrVitoria,pAtrDerrota )

if classe ==1:
    print('\n>>> Predicao = Vitoria!')       
else:
    print('\n>>> Predicao = Derrota!')

print('\n>>>>>> Prob. vitoria = %0.6f!' %(probVitoria))
print('\n>>>>>> Prob. derrota = %0.6f!\n\n'  %(probDerrota))


>>> Predicao = Derrota!

>>>>>> Prob. vitoria = 0.006996!

>>>>>> Prob. derrota = 0.024279!


